In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from pathlib import Path

import plotly.graph_objs as go
import plotly.express as px

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import (mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, silhouette_score, r2_score)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks
import matplotlib.pyplot as plt
import joblib

In [ ]:
df = pd.read_excel('data.xlsx', index_col='ID')
df.info


In [ ]:
df.sample(20)

In [ ]:
pd.set_option('display.max_rows', None)
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df['Угол нашивки, град'] = df['Угол нашивки, град'].astype(float)
df.info()

In [ ]:
for col in df.select_dtypes(include=['float64', 'int64']):
    plt.figure(figsize=(10, 6))
    sns.histplot(df[col], kde=True, bins=30, color='skyblue')
    plt.title(f'Распределение {col} с KDE')
    plt.xlabel(col)
    plt.ylabel('Частота')
    plt.show()

In [ ]:
# Расчет корреляции
corr = df.corr()

# Построение тепловой карты
plt.figure(figsize=(12, 10))
sns.heatmap(
    corr,
    annot=True,
    fmt=".2f",
    cmap="RdYlGn",
    linewidths=0.5,
    square=True
)
plt.title("Тепловая карта корреляции", fontsize=14)
plt.show()

In [ ]:
df.corr(method ='pearson')

In [ ]:
df.corr(method ='kendall')

In [ ]:
def find_outliers_iqr(data):
    outliers = pd.DataFrame()
    for col in data.select_dtypes(include=['number']): 
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        outliers[col] = ~data[col].between(lower, upper) 
    return outliers

outliers_iqr = find_outliers_iqr(df)
print("Выбросы по IQR:")
print(outliers_iqr.sum())  

In [ ]:
plt.figure(figsize=(24, 12))
sns.boxplot(data=df.select_dtypes(include=['number']))
plt.xticks(rotation=90)
plt.title("Выбросы по всем столбцам")
plt.show()

In [ ]:
def replace_outliers_iqr_with_mean(data):
    data_clean = data.copy()
    for col in data.select_dtypes(include=['number']):
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        mean_val = data[(data[col] >= lower_bound) & (data[col] <= upper_bound)][col].mean()
        data_clean[col] = np.where(
            (data[col] < lower_bound) | (data[col] > upper_bound),
            mean_val,
            data[col]
        )
    return data_clean

In [ ]:
(df.isnull().mean()).rename('доля выбросов').to_frame().style.bar(vmax=1, vmin=0).format('{:.2%}')

In [ ]:
cluster_features = ['Модуль упругости при растяжении, ГПа']

scaler = StandardScaler()
scaled_data = scaler.fit_transform(df[cluster_features])

wcss = []  
for i in range(1, 16):
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state=42)
    kmeans.fit(scaled_data)
    wcss.append(kmeans.inertia_)

plt.figure(figsize=(10, 6))
plt.plot(range(1, 16), wcss, marker='o', linestyle='--')
plt.title('Метод локтя для определения оптимального числа кластеров')
plt.xlabel('Число кластеров')
plt.ylabel('WCSS')
plt.show()

optimal_clusters = 6
kmeans = KMeans(n_clusters=optimal_clusters, init='k-means++', random_state=42)
clusters = kmeans.fit_predict(scaled_data)

df['Cluster'] = clusters

silhouette_avg = silhouette_score(scaled_data, clusters)
print(f"Средний коэффициент силуэта: {silhouette_avg:.3f}")

cluster_stats = df.groupby('Cluster')[cluster_features].mean()
print("\nСредние значения по кластерам:")
print(cluster_stats)

In [ ]:
df['Cluster'] = np.nan
df.loc[df.index, 'Cluster'] = clusters
print(f"Размер df: {len(df)}")
print(f"Уникальные кластеры: {df['Cluster'].unique()}")

In [ ]:
def evaluate_model(y_true, y_pred):
    print(f'MAE: {mean_absolute_error(y_true, y_pred):.4f}')
    print(f'R2: {r2_score(y_true, y_pred):.4f}')
    print(f'MSE: {mean_squared_error(y_true, y_pred):.4f}') 
    print(f'RMSE: {np.sqrt(mean_squared_error(y_true, y_pred)):.4f}')

In [ ]:
df['Cluster'] = clusters

X = df.drop(['Модуль упругости при растяжении, ГПа', 'Прочность при растяжении, МПа'], axis=1)
y = df['Модуль упругости при растяжении, ГПа']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [ ]:
print(len(X_train.columns))
X_train.head()

In [ ]:
model = GradientBoostingRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=7,
    random_state=42
)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
print("\nОценка модели на тестовых данных:")
evaluate_model(y_test, y_pred)

In [ ]:
data_to_check = 111
print(X_train.iloc[data_to_check])
print()
print(y_train.iloc[data_to_check])

In [ ]:
model.predict(X_test.iloc[data_to_check:data_to_check+1])

In [ ]:
X_test.insert(0, 'pred', y_pred) 
X_test.insert(9, 'Модуль упругости при растяжении, ГПа', y_pred)
X_test.insert(10, 'Прочность при растяжении, МПа', y_pred)
X_test.head()

In [ ]:
with open('model_1.sav', 'wb') as f:
    pickle.dump(model, f)
    
with open('model_1.sav', 'rb') as f: 
    model_1 = pickle.load(f)

In [ ]:
cluster_features = ['Прочность при растяжении, МПа']

scaler = StandardScaler()
scaled_data = scaler.fit_transform(df[cluster_features])

wcss = []  
for i in range(1, 16):
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state=42)
    kmeans.fit(scaled_data)
    wcss.append(kmeans.inertia_)

plt.figure(figsize=(10, 6))
plt.plot(range(1, 16), wcss, marker='o', linestyle='--')
plt.title('Метод локтя для определения оптимального числа кластеров')
plt.xlabel('Число кластеров')
plt.ylabel('WCSS')
plt.show()

optimal_clusters = 6
kmeans = KMeans(n_clusters=optimal_clusters, init='k-means++', random_state=42)
clusters = kmeans.fit_predict(scaled_data)

df['Cluster'] = clusters

silhouette_avg = silhouette_score(scaled_data, clusters)
print(f"Средний коэффициент силуэта: {silhouette_avg:.3f}")

cluster_stats = df.groupby('Cluster')[cluster_features].mean()
print("\nСредние значения по кластерам:")
print(cluster_stats)

In [ ]:
df['Cluster'] = np.nan
df.loc[df.index, 'Cluster'] = clusters
print(f"Размер df: {len(df)}")
print(f"Уникальные кластеры: {df['Cluster'].unique()}")

In [ ]:
def evaluate_model(y_true, y_pred):
    print(f'MAE: {mean_absolute_error(y_true, y_pred):.4f}')
    print(f'R2: {r2_score(y_true, y_pred):.4f}')
    print(f'MSE: {mean_squared_error(y_true, y_pred):.4f}') 
    print(f'RMSE: {np.sqrt(mean_squared_error(y_true, y_pred)):.4f}')

In [ ]:
df['Cluster'] = clusters

X = df.drop(['Модуль упругости при растяжении, ГПа', 'Прочность при растяжении, МПа'], axis=1)
y = df['Прочность при растяжении, МПа']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [ ]:
print(len(X_train.columns))
X_train.head()

In [ ]:
model = AdaBoostRegressor(
    estimator=DecisionTreeRegressor(max_depth=4),
    n_estimators=100,
    learning_rate=0.1,
    random_state=42
)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
print("\nОценка модели на тестовых данных:")
evaluate_model(y_test, y_pred)

In [ ]:
y_pred = model.predict(X_test) 

In [ ]:
data_to_check = 111
print(X_train.iloc[data_to_check])
print()
print(y_train.iloc[data_to_check])

In [ ]:
model.predict(X_test.iloc[data_to_check:data_to_check+1])

In [ ]:
X_test.insert(0, 'pred', y_pred) 
X_test.insert(9, 'Модуль упругости при растяжении, ГПа', y_pred)
X_test.insert(10, 'Прочность при растяжении, МПа', y_pred)
X_test.head()

In [ ]:
with open('model_2.sav', 'wb') as f:
    pickle.dump(model, f)
    
with open('model_2.sav', 'rb') as f: 
    model_2 = pickle.load(f)

In [ ]:
def load_and_prepare_data():
    input_features = [
        "Плотность, кг/м3", "модуль упругости, ГПа", 
        "Количество отвердителя, м.%", "Содержание эпоксидных групп,%_2",
        "Температура вспышки, С_2", "Поверхностная плотность, г/м2",
        "Прочность при растяжении, МПа", "Модуль упругости при растяжении, ГПа",
        "Потребление смолы, г/м2", "Угол нашивки, град", 
        "Шаг нашивки", "Плотность нашивки"
    ]
    target = 'Соотношение матрица-наполнитель'

    X = df[input_features]
    y = df[[target]]
    
    return X, y

In [ ]:
def build_model():
    model = keras.Sequential()
    
    # Фиксированная архитектура: 2 слоя по 8 нейронов
    model.add(layers.Dense(8, activation='relu', input_shape=(12,)))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(8, activation='relu'))
    model.add(layers.Dropout(0.2))
    
    model.add(layers.Dense(1))
    
    optimizer = keras.optimizers.Adam(learning_rate=0.001)
    
    model.compile(
        optimizer=optimizer,
        loss='mse',
        metrics=['mae', keras.metrics.MeanAbsolutePercentageError(name='mape')]
    )
    return model

In [ ]:
def train_with_cv(X, y, n_splits=5, epochs=100):
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    cv_scores = []
    histories = []
    
    for train_idx, val_idx in kfold.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        # Масштабирование
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_val_scaled = scaler.transform(X_val)
        
        model = build_model()
        
        early_stopping = callbacks.EarlyStopping(
            patience=10,
            restore_best_weights=True
        )
        
        history = model.fit(
            X_train_scaled, y_train,
            validation_data=(X_val_scaled, y_val),
            epochs=epochs,
            batch_size=32,
            callbacks=[early_stopping],
            verbose=0
        )
        
        val_loss, val_mae, val_mape = model.evaluate(X_val_scaled, y_val, verbose=0)
        cv_scores.append(val_mae)
        histories.append(history)
    
    return cv_scores, histories

In [ ]:
def main():
    # Загрузка данных
    X, y = load_and_prepare_data()
    
    # Кросс-валидация
    cv_scores, histories = train_with_cv(X, y)
    print(f"Average CV MAE: {np.mean(cv_scores):.4f} ± {np.std(cv_scores):.4f}")
    
    # Окончательное обучение на всех данных
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42
    )
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Создание и обучение модели с фиксированной архитектурой
    model = build_model()
    
    # Обучение модели с историей
    history = model.fit(
        X_train_scaled, y_train,
        validation_data=(X_test_scaled, y_test),
        epochs=100,
        batch_size=32,
        callbacks=[callbacks.EarlyStopping(patience=10, restore_best_weights=True)],
        verbose=1
    )
    
    # Визуализация функции потерь для финального обучения
    plt.figure(figsize=(12, 6))
    plt.plot(history.history['loss'], label='Потери обучения')
    plt.plot(history.history['val_loss'], label='Потери валидации')
    plt.title('Функции потерь финальной модели')
    plt.ylabel('Потери (MSE)')
    plt.xlabel('Эпохи')
    plt.legend()
    plt.grid(True)
    plt.show()
    
    # Оценка на тестовых данных
    test_loss, test_mae, test_mape = model.evaluate(X_test_scaled, y_test)
    print(f"Test MAE: {test_mae:.4f}")
    print(f"Test MAPE: {test_mape:.2f}%")
    
    # Прогнозирование
    y_pred = model.predict(X_test_scaled)
    
    # Визуализация результатов
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, y_pred, alpha=0.5)
    plt.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=2)
    plt.xlabel('Истинные значения')
    plt.ylabel('Предсказания')
    plt.title('Истинные vs предсказанные значения')
    plt.show()
    
    # Сохранение модели
    model.save('dnn_model.keras')
    
    # Важность признаков (пример)
    feature_importance = np.abs(model.layers[0].get_weights()[0]).mean(axis=1)
    plt.figure(figsize=(10, 6))
    plt.barh(X.columns, feature_importance)
    plt.title('Важность признаков')
    plt.show()

if __name__ == "__main__":
    main()

In [ ]:
def calculate_regression_metrics(y_true, y_pred):
    metrics = {
        'MAE': mean_absolute_error(y_true, y_pred),
        'MSE': mean_squared_error(y_true, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
        'R2': r2_score(y_true, y_pred),
        'MAPE (%)': mean_absolute_percentage_error(y_true, y_pred) * 100
    }
    return metrics

def print_metrics_table(metrics):
    """"""
    print("\nРегрессионные метрики:")
    print("{:<20} {:<10}".format('Метрика', 'Значение'))
    print("-" * 30)
    for name, value in metrics.items():
        print("{:<20} {:<10.4f}".format(name, value))

if __name__ == "__main__":
    y_true = df['Соотношение матрица-наполнитель'].values
    y_pred = df['Соотношение матрица-наполнитель'].values * 0.95
    
    # Вычисление метрик
    metrics = calculate_regression_metrics(y_true, y_pred)
    
    # Вывод результатов
    print_metrics_table(metrics)

# %%
model = keras.models.load_model('dnn_model.keras')
model.summary()